# Mapping ChEMBL to ChEBI

The `MOLECULE_DICTIONARY` table in ChEMBL contains mappings to ChEBI for some, but not all chemicals. This is unsurprising, given the scope of ChEBML is larger tha ChEBI. However, there is still room for improving these mappings.

This notebook identifies molecules that have no ChEBI mapping (and have a label) then generates prioritized lexical matchings usingy [`gilda`](https://github.com/indralab/gilda) for curation.

In [1]:
import sys
import time

import gilda.grounder
import gilda.term
import pandas as pd
from biomappings.gilda_utils import iter_prediction_tuples
from gilda.process import normalize
from tqdm.auto import tqdm
from IPython.display import Markdown

from chembl_downloader import queries, query, latest

In [2]:
print(sys.version)

3.10.8 (main, Oct 13 2022, 10:17:43) [Clang 14.0.0 (clang-1400.0.29.102)]


In [3]:
print(time.asctime())

Fri Oct 28 15:16:09 2022


In [4]:
version = latest()
print(f"Using ChEMBL version {version}")

Using ChEMBL version 31


## Making the Query

The following query over the `MOLECULE_DICTIONARY` finds all ChEMBL compound identifiers and their associated preferred names but filters out ones that already have mappings to ChEBI. This allows us to focus on doing some extra curation of new mappings.

In [5]:
queries.markdown(queries.CHEBI_UNMAPPED_SQL)

```sql
SELECT
    chembl_id,
    pref_name
FROM MOLECULE_DICTIONARY
WHERE 
    chebi_par_id IS NULL
    AND pref_name IS NOT NULL
```

Make the query with `chembl_downloader.query`.

In [6]:
%time
df = query(queries.CHEBI_UNMAPPED_SQL, version=version)

CPU times: user 2 µs, sys: 1e+03 ns, total: 3 µs
Wall time: 5.25 µs


In [7]:
df

,chembl_id,pref_name
0,CHEMBL6206,BROMOENOL LACTONE
1,CHEMBL446445,UCL-1530
2,CHEMBL266459,ZOMEPIRAC SODIUM
3,CHEMBL216458,ALPHA-BUNGAROTOXIN
4,CHEMBL6346,SCR01020
...,...,...
39225,CHEMBL4802269,IZURALIMAB
39226,CHEMBL4802270,PUDEXACIANINIUM
39227,CHEMBL4804171,AFP-464 FREE SALT
39228,CHEMBL4804172,SAMARIUM DOTMP


## Propose New Mappings

First, we index the dataframe of molecules using [`gilda`](https://github.com/indralab/gilda), which implements a scored string matching algorithm.

In [8]:
terms = [
    gilda.term.Term(
        norm_text=normalize(name),
        text=name,
        db="chembl.compound",
        id=identifier,
        entry_name=name,
        status="name",
        source="chembl",
    )
    for identifier, name in tqdm(df.values, unit="term", unit_scale=True)
]

grounder = gilda.grounder.Grounder(terms)

  0%|          | 0.00/39.2k [00:00<?, ?term/s]

Second, we use a utility function from [`biomappings`](https://github.com/biopragmatics/biomappings) that takes in three things:

1. a `prefix` corresponding to the resource we want to map against
2. the `grounder` object generated from indexing all of the ChEMBL terms
3. a `provenance` string


This function in turn relies on [`pyobo`](https://github.com/pyobo/pyobo) and will download/cache the [ChEBI ontology](https://obofoundry.org/ontology/chebi), so be patient on the first run.

In [9]:
prediction_tuples = list(
    iter_prediction_tuples(
        prefix="chebi",
        grounder=grounder,
        provenance="notebook",
    )
)

[chebi] gilda tuples:   0%|          | 0.00/163k [00:00<?, ?name/s]

In [10]:
print(f"Got {len(prediction_tuples):,} predictions")

Got 4,720 predictions


## Results

The results below show promising results, often resulting in exact string matches. Further proofing can be done on the chemical strucutre level, but these matches are typically correct without further investigation.

In [11]:
predictions_df = pd.DataFrame(prediction_tuples).sort_values("confidence", ascending=False)
predictions_df

,source_prefix,source_id,source_name,relation,target_prefix,target_identifier,target_name,type,confidence,source
4377,chebi,CHEBI:80180,Indolicidin,skos:exactMatch,chembl.compound,CHEMBL2251917,Indolicidin,lexical,0.777778,notebook
3524,chebi,CHEBI:40701,ALRESTATIN,skos:exactMatch,chembl.compound,CHEMBL63055,ALRESTATIN,lexical,0.777778,notebook
1765,chebi,CHEBI:167900,PJ34,skos:exactMatch,chembl.compound,CHEMBL3233481,PJ34,lexical,0.777778,notebook
1766,chebi,CHEBI:167900,PJ34,skos:exactMatch,chembl.compound,CHEMBL372303,PJ34,lexical,0.777778,notebook
3520,chebi,CHEBI:40287,7-NITROINDAZOLE,skos:exactMatch,chembl.compound,CHEMBL247378,7-NITROINDAZOLE,lexical,0.777778,notebook
...,...,...,...,...,...,...,...,...,...,...
4662,chebi,CHEBI:9423,Technetium tc 99m sestamibi,skos:exactMatch,chembl.compound,CHEMBL4594241,TECHNETIUM TC 99M SESTAMIBI,lexical,0.723974,notebook
3310,chebi,CHEBI:32187,Technetium Tc 99m succimer,skos:exactMatch,chembl.compound,CHEMBL1200797,TECHNETIUM TC 99M SUCCIMER,lexical,0.723974,notebook
3774,chebi,CHEBI:5937,Interferon alfa-2a,skos:exactMatch,chembl.compound,CHEMBL2108508,INTERFERON ALFA-2A,lexical,0.723974,notebook
3775,chebi,CHEBI:5938,Interferon beta-1b,skos:exactMatch,chembl.compound,CHEMBL1201563,INTERFERON BETA-1B,lexical,0.723974,notebook
